In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
import hvplot.pandas
import os
import requests
import datetime
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.requests import CryptoLatestQuoteRequest
from alpaca.data.requests import CryptoTradesRequest
from alpaca.data.timeframe import TimeFrame

In [9]:
load_dotenv()

True

In [10]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [11]:
type(alpaca_api_key)

str

In [12]:
crypto_client = CryptoHistoricalDataClient(alpaca_api_key, alpaca_secret_key)

In [13]:
request_params = CryptoBarsRequest(
    symbol_or_symbols=["BTC/USD"],
    timeframe=TimeFrame.Day,
    start="2012-01-01 00:00:00"
)

btc_bars = crypto_client.get_crypto_bars(request_params)

btc_df = btc_bars.df

In [14]:
btc_df = btc_df.reset_index()

In [15]:
btc_df = btc_df.set_index('timestamp')

In [16]:
btc_df['close'].hvplot()

:Curve   [timestamp]   (close)

In [154]:
# Filter the date index and close columns
signals_df = btc_df[['close']]
signals_df

# Set the short window and long windows
short_window = 20
long_window = 50

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA_Short'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Long'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA_Short'][short_window:] > signals_df['SMA_Long'][short_window:], 1.0, 0.0
)

# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

C:\Anaconda\envs\dev\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda\envs\dev\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,close,SMA_Short,SMA_Long,Signal,Entry/Exit
timestamp,,,,,
2022-09-25 05:00:00+00:00,18811.0,19895.40,21097.30,0.0,0.0
2022-09-26 05:00:00+00:00,20048.0,19961.80,21031.62,0.0,0.0
2022-09-27 05:00:00+00:00,18765.0,19932.50,20929.16,0.0,0.0
2022-09-28 05:00:00+00:00,19602.0,19894.30,20862.12,0.0,0.0
2022-09-29 05:00:00+00:00,19432.0,19793.45,20760.14,0.0,0.0
2022-09-30 05:00:00+00:00,19410.0,19685.70,20669.46,0.0,0.0
2022-10-01 05:00:00+00:00,19331.0,19566.30,20561.18,0.0,0.0
2022-10-02 05:00:00+00:00,19099.0,19406.90,20451.58,0.0,0.0
2022-10-03 05:00:00+00:00,19603.0,19367.90,20346.40,0.0,0.0


In [155]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA_Short', 'SMA_Long']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="btc - SMAShort, SMALong, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [timestamp]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [156]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 1

In [157]:
# Buy a share position when the dual moving average crossover Signal equals 1
# Sell a share position when the dual moving average crossover Signal equals 0
signals_df['Position'] = share_size * signals_df['Signal']

C:\Anaconda\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
# Determine the points in time where a share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

C:\Anaconda\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [159]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

In [160]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

In [161]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

In [162]:
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

In [163]:
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

In [164]:
# Print the DataFrame
signals_df

,close,SMA_Short,SMA_Long,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
timestamp,,,,,,,,,,,,
2020-04-08 05:00:00+00:00,7314.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2020-04-14 05:00:00+00:00,7100.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,NaN,NaN
2020-04-22 05:00:00+00:00,7118.5,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
2020-04-28 05:00:00+00:00,7836.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
2020-04-30 05:00:00+00:00,8653.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 05:00:00+00:00,19410.0,19685.7,20669.46,0.0,0.0,0.0,0.0,0.0,132569.5,132569.5,0.0,0.325695
2022-10-01 05:00:00+00:00,19331.0,19566.3,20561.18,0.0,0.0,0.0,0.0,0.0,132569.5,132569.5,0.0,0.325695
2022-10-02 05:00:00+00:00,19099.0,19406.9,20451.58,0.0,0.0,0.0,0.0,0.0,132569.5,132569.5,0.0,0.325695


In [165]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [timestamp]   (Portfolio Total)

In [166]:
# Create a DataFrame that will hold portfolio evaluation metrics

# Create a list for the column name
columns = ['Backtest']

# Create a list holding the names of the new evaluation metrics
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [167]:
# Calculate annualized return (average daily return multipled by 252 trading days in a year)
portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.088832
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [168]:
# Calculate cumulative return (Last value of Portfolio Cumulative Returns Column)
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annualized Return,0.088832
Cumulative Returns,0.325695
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [169]:
# Calculate annual volatility (std dev of the daily returns column multiplied by the square root of 252 trading days)
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.088832
Cumulative Returns,0.325695
Annual Volatility,0.125806
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [170]:
# Calculate Sharpe ratio (Annualized Return / Annual Volatility)
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.088832
Cumulative Returns,0.325695
Annual Volatility,0.125806
Sharpe Ratio,0.706099
Sortino Ratio,NaN


In [171]:
# Calculate downside return values (square asset's negative daily return values and then calculate the annualized return value of the asset)

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
timestamp,,
2022-09-30 05:00:00+00:00,0.0,0.0
2022-10-01 05:00:00+00:00,0.0,0.0
2022-10-02 05:00:00+00:00,0.0,0.0
2022-10-03 05:00:00+00:00,0.0,0.0
2022-10-04 05:00:00+00:00,0.0,0.0


In [172]:
# Calculate the Sortino ratio

# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
)

# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annualized Return,0.088832
Cumulative Returns,0.325695
Annual Volatility,0.125806
Sharpe Ratio,0.706099
Sortino Ratio,1.044297


In [173]:
# Create a DataFrame that holds the evaluation metrics per trade

# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', # The name of the asset that we’re trading.
        'Entry Date', # The date that we entered (bought) the trade.
        'Exit Date', # The date that we exited (sold) the trade.
        'Shares', # The number of shares that we executed for the trade.
        'Entry Share Price', # The price of the asset when we entered the trade.
        'Exit Share Price', # he price of the asset when we exited the trade.
        'Entry Portfolio Holding', # The cost of the trade on entry (which is the number of shares multiplied by the entry share price)
        'Exit Portfolio Holding', # The proceeds that we made from the trade on exit (which is the number of shares multiplied by the exit share price)
        'Profit/Loss'] # The profit or loss from the trade (which is the proceeds from the trade minus the cost of the trade)
)

In [174]:
# Initialize iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0


In [175]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2020-06-21 05:00:00+00:00,2020-06-30 05:00:00+00:00,1.0,9394.5,9110.5,9394.5,9110.5,-284.0
1,AAPL,2020-07-26 05:00:00+00:00,2020-09-11 05:00:00+00:00,1.0,10307.5,10347.5,10307.5,10347.5,40.0
2,AAPL,2020-10-13 05:00:00+00:00,2021-04-28 05:00:00+00:00,1.0,11407.5,53634.0,11407.5,53634.0,42226.5
3,AAPL,2021-08-01 05:00:00+00:00,2021-09-23 05:00:00+00:00,1.0,39704.0,44287.0,39704.0,44287.0,4583.0
4,AAPL,2021-10-11 05:00:00+00:00,2021-11-27 06:00:00+00:00,1.0,56956.0,54419.0,56956.0,54419.0,-2537.0
5,AAPL,2022-02-18 06:00:00+00:00,2022-03-07 06:00:00+00:00,1.0,40255.0,38345.0,40255.0,38345.0,-1910.0
6,AAPL,2022-03-19 05:00:00+00:00,2022-03-20 05:00:00+00:00,1.0,41856.0,40917.0,41856.0,40917.0,-939.0
7,AAPL,2022-03-26 05:00:00+00:00,2022-04-22 05:00:00+00:00,1.0,44599.0,39544.0,44599.0,39544.0,-5055.0
8,AAPL,2022-07-28 05:00:00+00:00,2022-08-29 05:00:00+00:00,1.0,24019.0,20464.0,24019.0,20464.0,-3555.0


In [176]:
cp = trade_evaluation_df['Profit/Loss'].sum()
cp

32569.5